# ML model deployment via mlflow using fastapi as web service

## Introduction to MLflow Model Deployment

MLflow is an open-source platform that extends beyond experiment tracking to model deployment and serving. This lab demonstrates how to train a machine learning model, register it
in MLflow's Model Registry, and deploy it as a web service using FastAPI for real-time predictions.

## Why Deploy Models with MLflow and FastAPI?
MLflow combined with FastAPI simplifies the machine learning deployment process by:

- Model Registry: Centralized model versioning and management
- RESTful API: Easy integration with web applications and services
- Reproducibility: Consistent model deployment across environments
- Scalability: Production-ready API endpoints for real-time inference

This notebook demonstrates these capabilities using a telecom churn dataset with a Random Forest classifier, covering model training, registration, and deployment as a web service.

## Setup
First, let's install the required packages and mount Google Drive.

In [1]:
# Install dependencies
!pip install mlflow pyngrok fastapi uvicorn scikit-learn pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries

In [3]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import uvicorn
import threading
import subprocess
import time
from pyngrok import ngrok
import requests
import json




## Data Preparation
### The dataset contains telecom customer information for churn prediction. We'll preprocess the data to make it suitable for modeling.



In [5]:
# Load and preprocess data
df = pd.read_csv("/content/drive/MyDrive/cognixia_labs_test/Week_2_labs/Lab_3 Streamlit UI for model and fastapi service/telecome data.csv")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

feature_columns = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
                   'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
                   'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
                   'MonthlyCharges', 'TotalCharges']
X = df[feature_columns].copy()
y = df['Churn'].copy()

# Encode categorical variables
label_encoders = {}
categorical_columns = X.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Encode target variable
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


/tmp/ipython-input-5-1083194028.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)


## Model Training and Registration
### Train a Random Forest model and register it in MLflow's Model Registry for version control.

In [6]:
# Set MLflow tracking URI
mlflow.set_tracking_uri("file:///content/mlruns")

# Train and register model
with mlflow.start_run(run_name="churn_model"):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    mlflow.sklearn.log_model(model, "random_forest_model")
    accuracy = model.score(X_test, y_test)
    mlflow.log_metric("accuracy", accuracy)
    model_uri = f"runs:/{mlflow.active_run().info.run_id}/random_forest_model"
    registered_model = mlflow.register_model(model_uri, "ChurnModel")

print(f"Model registered: {registered_model.name}, Version: {registered_model.version}")

2025/07/08 10:14:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/08 10:14:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ChurnModel'.
2025/07/08 10:14:56 WARNING mlflow.tracking._model_registry.fluent: Run with id 2da6c0fe69ab42b780c4e27b3c2fa1ea has no artifacts at artifact path 'random_forest_model', registering model based on models:/m-0168104dab2c4d93b0b58027d643676d instead


Model registered: ChurnModel, Version: 1


Created version '1' of model 'ChurnModel'.


### Explanation:

- mlflow.set_tracking_uri(): Sets local file system as MLflow backend
- mlflow.sklearn.log_model(): Saves the trained model as an artifact
- mlflow.register_model(): Registers the model in the Model Registry for version control

## MLflow UI Setup
### Launch the MLflow UI to visualize experiments and manage models.

In [7]:
# Start MLflow UI server
mlflow_ui_command = "mlflow ui --backend-store-uri file:///content/mlruns --host 0.0.0.0 --port 5001 &"
subprocess.Popen(mlflow_ui_command, shell=True)
time.sleep(5)

## FastAPI Web Service
### Create a REST API endpoint for model predictions using FastAPI.

In [8]:
# FastAPI setup
app = FastAPI()

class PredictionInput(BaseModel):
    data: List[List[float]]

model = mlflow.sklearn.load_model(f"models:/{registered_model.name}/{registered_model.version}")

@app.post("/predict")
async def predict(input_data: PredictionInput):
    try:
        input_df = pd.DataFrame(input_data.data, columns=feature_columns)
        for col in categorical_columns:
            input_df[col] = input_df[col].astype(int)
        predictions = model.predict(input_df)
        predictions = target_encoder.inverse_transform(predictions)
        return {"predictions": predictions.tolist()}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

## Explanation:

- PredictionInput: Pydantic model for input validation
- mlflow.sklearn.load_model(): Loads the registered model from MLflow
- @app.post("/predict"): Creates a POST endpoint for predictions
- target_encoder.inverse_transform(): Converts predictions back to original labels

## Deployment with Ngrok
### Deploy both FastAPI and MLflow UI with public URLs using ngrok.

In [9]:
# Start FastAPI server in a thread
def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=5000)

threading.Thread(target=run_fastapi, daemon=True).start()
time.sleep(5)

# Expose FastAPI with ngrok
ngrok_token = "2yGBZnB7ngE0P19IGz2Qf2DW2EW_3Bj4VstSQQiy16UdWqKvX"
ngrok.set_auth_token(ngrok_token)
ngrok.kill()
public_url = ngrok.connect(5000, bind_tls=True).public_url
print(f"FastAPI server is exposed at: {public_url}")

# Expose MLflow UI with ngrok
mlflow_public_url = ngrok.connect(5001, bind_tls=True).public_url
print(f"MLflow UI is exposed at: {mlflow_public_url}")



INFO:     Started server process [227]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


FastAPI server is exposed at: https://d8ca58d571a4.ngrok-free.app
MLflow UI is exposed at: https://28e955844dac.ngrok-free.app


## Explanation:

- threading.Thread(): Runs FastAPI server in background
- ngrok.connect(): Creates secure public URLs for both services
- bind_tls=True: Enables HTTPS for secure connections

## Test the deployed model with sample data.

In [10]:
# Test the FastAPI endpoint
sample_data = {
    "data": [
        [1, 0, 1, 0, 34, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 56.95, 1889.5]
    ]
}
response = requests.post(f"{public_url}/predict", json=sample_data)
print(response.json())

INFO:     34.87.93.56:0 - "POST /predict HTTP/1.1" 200 OK
{'predictions': ['No']}


## Explanation:

- Creates sample input data matching the expected feature format
- Sends POST request to the prediction endpoint
- Returns the model's churn prediction for the customer

### This lab demonstrates a complete MLOps workflow from model training to deployment, showing how MLflow and FastAPI work together to create production-ready machine learning services.